In [3]:
# GitHub Repository Statistics Viewer
# This notebook displays the collected repository statistics in interactive tables

import pandas as pd
import json
import os
from datetime import datetime
from IPython.display import HTML, display, Markdown

# Check if data exists
if not os.path.exists('github_repository_stats.csv'):
    print("⚠️  No data found! Please run github_data_collector.ipynb first to collect data.")
else:
    # Load data
    df = pd.read_csv('github_repository_stats.csv')
    
    # Load metadata if available
    metadata = {}
    if os.path.exists('github_stats_metadata.json'):
        with open('github_stats_metadata.json', 'r') as f:
            metadata = json.load(f)
    
    # Display header
    display(Markdown("# 📊 GitHub Repository Statistics\n"))
    
    if metadata:
        last_updated = datetime.fromisoformat(metadata['last_updated']).strftime('%Y-%m-%d %H:%M')
        display(Markdown(f"**Last Updated:** {last_updated} | **Total Repositories:** {metadata['total_repositories']} | **Total Stars:** {metadata['total_stars']:,}"))
    
    # Quick summary stats
    display(Markdown("\n## 📈 Summary Statistics\n"))
    summary_html = f"""
    <div style="display: flex; gap: 20px; margin-bottom: 30px;">
        <div style="background: #f8f9fa; padding: 15px; border-radius: 8px; flex: 1;">
            <h4 style="margin: 0; color: #495057;">Total Stars</h4>
            <p style="font-size: 24px; font-weight: bold; margin: 5px 0; color: #212529;">{df['Stars'].sum():,}</p>
        </div>
        <div style="background: #f8f9fa; padding: 15px; border-radius: 8px; flex: 1;">
            <h4 style="margin: 0; color: #495057;">Total Forks</h4>
            <p style="font-size: 24px; font-weight: bold; margin: 5px 0; color: #212529;">{df['Forks'].sum():,}</p>
        </div>
        <div style="background: #f8f9fa; padding: 15px; border-radius: 8px; flex: 1;">
            <h4 style="margin: 0; color: #495057;">Avg Stars per Repo</h4>
            <p style="font-size: 24px; font-weight: bold; margin: 5px 0; color: #212529;">{df['Stars'].mean():.0f}</p>
        </div>
        <div style="background: #f8f9fa; padding: 15px; border-radius: 8px; flex: 1;">
            <h4 style="margin: 0; color: #495057;">Total Open Issues</h4>
            <p style="font-size: 24px; font-weight: bold; margin: 5px 0; color: #212529;">{df['Open Issues'].sum():,}</p>
        </div>
    </div>
    """
    display(HTML(summary_html))
    
    # Display tables by field
    field_order = ['AI/ML', 'TypeScript', 'C# ASP.NET', 'Other']
    
    for field in field_order:
        if field in df['Field'].values:
            field_df = df[df['Field'] == field].sort_values('Stars', ascending=False)
            
            display(Markdown(f"\n## {field} Repositories ({len(field_df)} total)\n"))
            
            # Create clickable links
            display_df = field_df.copy()
            display_df['Name'] = display_df['Name'].apply(
                lambda x: f'<a href="https://github.com/{x}" target="_blank" style="color: #0366d6; text-decoration: none;">{x}</a>'
            )
            
            # Format numbers with commas
            display_df['Stars'] = display_df['Stars'].apply(lambda x: f'{x:,}')
            display_df['Forks'] = display_df['Forks'].apply(lambda x: f'{x:,}')
            display_df['Open Issues'] = display_df['Open Issues'].apply(lambda x: f'{x:,}')
            
            # Drop Field column for cleaner display
            display_df = display_df.drop('Field', axis=1)
            
            # Custom table styling
            html_table = display_df.to_html(escape=False, index=False)
            
            styled_table = f"""
            <style>
                .repo-table {{
                    border-collapse: collapse;
                    width: 100%;
                    margin-bottom: 30px;
                    font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
                }}
                .repo-table th {{
                    background-color: #f6f8fa;
                    border: 1px solid #e1e4e8;
                    padding: 10px 12px;
                    text-align: left;
                    font-weight: 600;
                    color: #24292e;
                }}
                .repo-table td {{
                    border: 1px solid #e1e4e8;
                    padding: 8px 12px;
                    color: #24292e;
                }}
                .repo-table tr:hover {{
                    background-color: #f6f8fa;
                }}
                .repo-table a:hover {{
                    text-decoration: underline;
                }}
            </style>
            <div class="table-container">
                {html_table.replace('<table', '<table class="repo-table"')}
            </div>
            """
            
            display(HTML(styled_table))
    
    # Top repositories section
    display(Markdown("\n## 🏆 Top 10 Most Starred Repositories\n"))
    top_10 = df.nlargest(10, 'Stars')[['Name', 'Field', 'Stars', 'Forks']].copy()
    top_10['Rank'] = range(1, 11)
    top_10 = top_10[['Rank', 'Name', 'Field', 'Stars', 'Forks']]
    
    # Create bar chart visualization
    top_10_display = top_10.copy()
    top_10_display['Name'] = top_10_display['Name'].apply(
        lambda x: f'<a href="https://github.com/{x}" target="_blank" style="color: #0366d6; text-decoration: none;">{x}</a>'
    )
    
    # Add star bars
    max_stars = top_10['Stars'].max()
    top_10_display['Popularity'] = top_10['Stars'].apply(
        lambda x: f'<div style="background: #1f77b4; height: 20px; width: {(x/max_stars)*200}px; border-radius: 3px;"></div>'
    )
    
    top_10_display['Stars'] = top_10_display['Stars'].apply(lambda x: f'{x:,} ⭐')
    top_10_display['Forks'] = top_10_display['Forks'].apply(lambda x: f'{x:,}')
    
    top_table_html = top_10_display.to_html(escape=False, index=False)
    styled_top_table = f"""
    <div class="table-container">
        {top_table_html.replace('<table', '<table class="repo-table"')}
    </div>
    """
    display(HTML(styled_top_table))
    
    # Instructions for refreshing data
    display(Markdown("\n---\n**To refresh data:** Run `github_data_collector.ipynb` to fetch the latest statistics from GitHub."))

# 📊 GitHub Repository Statistics



## 📈 Summary Statistics



## AI/ML Repositories (20 total)


Name,Stars,Forks,Contributors,Open Issues,Open Pull Requests,Date Created,Last Active
tensorflow/tensorflow,"190,933","74,769",1,"1,510",1,2015-11-07,2025-07-28
Significant-Gravitas/AutoGPT,"177,287","45,904",1,212,1,2023-03-16,2025-07-28
ollama/ollama,"147,790","12,552",1,"1,957",1,2023-06-26,2025-07-27
huggingface/transformers,"147,573","29,808",1,"1,897",1,2018-10-29,2025-07-28
n8n-io/n8n,"124,448","37,748",1,983,1,2019-06-22,2025-07-28
langchain-ai/langchain,"112,412","18,351",1,277,1,2022-10-17,2025-07-28
langgenius/dify,"108,749","16,549",1,761,1,2023-04-12,2025-07-28
open-webui/open-webui,"104,200","14,012",1,286,1,2023-10-06,2025-07-28
pytorch/pytorch,"91,835","24,787",1,"16,649",1,2016-08-13,2025-07-28
comfyanonymous/ComfyUI,"83,714","9,284",1,"2,603",1,2023-01-17,2025-07-28



## TypeScript Repositories (21 total)


...


## Other Repositories (1 total)


Name,Stars,Forks,Contributors,Open Issues,Open Pull Requests,Date Created,Last Active
karpathy/nn-zero-to-hero,"15,105","2,120",1,43,1,2022-09-08,2024-08-18



## 🏆 Top 10 Most Starred Repositories


Rank,Name,Field,Stars,Forks,Popularity
1,tensorflow/tensorflow,AI/ML,"190,933 ⭐","74,769",
2,Significant-Gravitas/AutoGPT,AI/ML,"177,287 ⭐","45,904",
3,ollama/ollama,AI/ML,"147,790 ⭐","12,552",
4,huggingface/transformers,AI/ML,"147,573 ⭐","29,808",
5,n8n-io/n8n,AI/ML,"124,448 ⭐","37,748",
6,langchain-ai/langchain,AI/ML,"112,412 ⭐","18,351",
7,langgenius/dify,AI/ML,"108,749 ⭐","16,549",
8,open-webui/open-webui,AI/ML,"104,200 ⭐","14,012",
9,pytorch/pytorch,AI/ML,"91,835 ⭐","24,787",
10,comfyanonymous/ComfyUI,AI/ML,"83,714 ⭐","9,284",



---
**To refresh data:** Run `github_data_collector.ipynb` to fetch the latest statistics from GitHub.